<a href="https://colab.research.google.com/github/LIAORONGTIAN/Cookbook/blob/master/AdalFlow_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🤗 Generator

# Quick Links

Github repo: https://github.com/SylphAI-Inc/AdalFlow

Full Tutorials: https://adalflow.sylph.ai/index.html#

Discord: https://discord.gg/ezzszrRZvT

⭐ <i>Star us on <a href="https://github.com/SylphAI-Inc/AdalFlow">Github</a> </i> ⭐

## 📖 Outline

This is the code for tutorial: https://adalflow.sylph.ai/tutorials/generator.html

It covers:
* how to use generator with template, id, cache, and output parsing.

* how to switch to different model provider.
* how to create a Generator purely from json or yaml config.
* A simple chatbot.

In [ ]:
from IPython.display import clear_output

!pip install -U adalflow[groq]

clear_output()

Setup api key

In [ ]:
import os

from getpass import getpass

groq_api_key = getpass("Please enter your GROQ API key: ")


os.environ['GROQ_API_KEY'] = groq_api_key

print("API keys have been set.")

Please enter your GROQ API key: ··········
API keys have been set.


### Generator comes with default template

All we need is a ``model_client`` and a ``model_kwargs`` where the parameters are directly passed to the API provider as it is.

In default, generator comes with default template. You can visualize it with ``print(generator)``.

Note: `print(component)` is a great way to visualize its structure.

In [ ]:
import adalflow as adal
from adalflow.components.model_client import GroqAPIClient

generator = adal.Generator(
    model_client=GroqAPIClient(),
    model_kwargs={"model": "llama3-8b-8192"},
)

# print the generator structure
print(generator)

cache_path: /root/.adalflow/cache_GroqAPIClient_llama3-8b-8192.db
Generator(
  model_kwargs={'model': 'llama3-8b-8192'}, 
  (prompt): Prompt(
    template: <START_OF_SYSTEM_PROMPT>
    {# task desc #}
    {% if task_desc_str %}
    {{task_desc_str}}
    {% else %}
    You are a helpful assistant.
    {% endif %}
    {#input format#}
    {% if input_format_str %}
    <INPUT_FORMAT>
    {{input_format_str}}
    </INPUT_FORMAT>
    {% endif %}
    {# output format #}
    {% if output_format_str %}
    <OUTPUT_FORMAT>
    {{output_format_str}}
    </OUTPUT_FORMAT>
    {% endif %}
    {# tools #}
    {% if tools_str %}
    <TOOLS>
    {{tools_str}}
    </TOOLS>
    {% endif %}
    {# example #}
    {% if examples_str %}
    <EXAMPLES>
    {{examples_str}}
    </EXAMPLES>
    {% endif %}
    {# chat history #}
    {% if chat_history_str %}
    <CHAT_HISTORY>
    {{chat_history_str}}
    </CHAT_HISTORY>
    {% endif %}
    {#contex#}
    {% if context_str %}
    <CONTEXT>
    {{context_str}}



The default prompt is built on [Jinja2](https://jinja.palletsprojects.com/en/3.1.x/). We've included why using Jinja2 [here](https://lightrag.sylph.ai/tutorials/prompt.html#why-jinja2).


Now let's check the the final prompt with prompt variables:

In [ ]:
# you can use get_prompt or print_prompt

prompt_kwargs = {"input_str": "What is LLM? Explain in one sentence."}
generator.get_prompt(**prompt_kwargs)

'<START_OF_SYSTEM_PROMPT>\nYou are a helpful assistant.\n<END_OF_SYSTEM_PROMPT>\n<START_OF_USER_PROMPT>\nWhat is LLM? Explain in one sentence.\n<END_OF_USER_PROMPT>\n'

Here we only add the `input_str` so the prompt integrates the user query only, with the default system promp `You are a helpful assistant`.

Next, let's call the generator:

In [ ]:
output = generator(
    prompt_kwargs=prompt_kwargs,
)

print(output)

GeneratorOutput(id=None, data='LLM stands for Large Language Model, which refers to a type of artificial intelligence model designed to process and generate human-like language, often used in applications such as language translation, text summarization, and chatbots.', error=None, usage=CompletionUsage(completion_tokens=44, prompt_tokens=54, total_tokens=98), raw_response='LLM stands for Large Language Model, which refers to a type of artificial intelligence model designed to process and generate human-like language, often used in applications such as language translation, text summarization, and chatbots.', metadata=None)


### Generator with id

Optionally, you can pass an id to keep track of your LLM call.

In [ ]:
output = generator(
    prompt_kwargs=prompt_kwargs, id ="1",
)

print(output)

GeneratorOutput(id='1', data='LLM (Large Language Model) refers to a type of artificial intelligence designed to process and generate human-like language, using complex algorithms to analyze and understand vast amounts of text data.', error=None, usage=CompletionUsage(completion_tokens=37, prompt_tokens=54, total_tokens=91), raw_response='LLM (Large Language Model) refers to a type of artificial intelligence designed to process and generate human-like language, using complex algorithms to analyze and understand vast amounts of text data.', metadata=None)


In [ ]:
print(f"data: {output.data}")

data: LLM (Large Language Model) refers to a type of artificial intelligence designed to process and generate human-like language, using complex algorithms to analyze and understand vast amounts of text data.


### Generator output structure ``GeneratorOutput``

The output will be the GeneratorOutput object. It has:
- id (str)
- data (object)
- error (str)
- raw_response (str)
- metadata (dict)
- usage

This attributes make the usage of LLM output easier.

Whether to do further processing or terminate the pipeline whenever an error occurs is up to the user from here on.

### Generator with Customized Template

Let's customize the prompt here. We will use two variables `task_desc_str` and `input_str` in the prompt.

In [ ]:
template = r"""<START_OF_SYSTEM_PROMPT>{{task_desc_str}}<END_OF_SYSTEM_PROMPT>
<START_OF_USER_PROMPT>{{input_str}}<END_OF_USER_PROMPT>"""

generator = adal.Generator(
    model_client=GroqAPIClient(),
    model_kwargs={"model": "llama3-8b-8192"},
    template=template,
    prompt_kwargs={"task_desc_str": "You are a professional AI engineer."},
)

prompt_kwargs = {"input_str": "Explain what is LLM to a non professional."}

generator.get_prompt(
    **prompt_kwargs,
)

cache_path: /root/.adalflow/cache_GroqAPIClient_llama3-8b-8192.db


'<START_OF_SYSTEM_PROMPT>You are a professional AI engineer.<END_OF_SYSTEM_PROMPT>\n<START_OF_USER_PROMPT>Explain what is LLM to a non professional.<END_OF_USER_PROMPT>'

Now you will see the final prompt. To customize your own prompt, please stick to [Jinja2](https://jinja.palletsprojects.com/en/3.1.x/) syntax.

In [ ]:
output = generator(
    prompt_kwargs=prompt_kwargs,
)
print(output.data)

I'd be happy to explain Large Language Model (LLM) in simple terms!

Imagine you have a super intelligent conversational partner who can understand and respond to any question you throw at it. This person is like a human expert, but instead of being a flesh-and-blood entity, it's a computer program designed to mimic human-like conversations.

That's basically what an LLM is! It's a type of artificial intelligence (AI) that's trained on a massive amount of text data (think books, articles, social media posts, etc.). This training enables the LLM to learn patterns, relationships, and even underlying meanings in language.

When you ask an LLM a question or provide some input, it uses this learned knowledge to generate a response that's often surprisingly accurate and natural-sounding. It's like having a conversational AI sidekick that can help with tasks, answer questions, or even engage in witty banter!

Some common applications of LLMs include:

1. Virtual assistants: Imagine asking you

### Cache Generator

In cases, especially when training with auto-optimizers, we want to cache calls to save both time and cost. You can turn on the ``use_cache`` in the generator.

In default, AdalFlow will create a path at ~/.adalflow. Depends on your Os, you might have a different path.

In [ ]:
# check the cache path

from adalflow.utils.global_config import get_adalflow_default_root_path

get_adalflow_default_root_path()

'/root/.adalflow'

In [ ]:
generator = adal.Generator(
    model_client=GroqAPIClient(),
    model_kwargs={"model": "llama3-8b-8192"},
    template=template,
    prompt_kwargs={"task_desc_str": "You are a professional AI engineer."},
    use_cache=True,
)

cache_path: /root/.adalflow/cache_GroqAPIClient_llama3-8b-8192.db


In [ ]:
output = generator(
    prompt_kwargs=prompt_kwargs,
)
print(output.data)

Large Language Model (LLM) refers to a type of artificial intelligence model that is specifically designed to analyze, understand, and generate human-like text, often using vast amounts of training data to improve its language processing capabilities.


In [ ]:
# when we run this query again, the response is still same as it is cached.
output = generator(
    prompt_kwargs=prompt_kwargs,
)
print(output.data)

Large Language Model (LLM) refers to a type of artificial intelligence model that is specifically designed to analyze, understand, and generate human-like text, often using vast amounts of training data to improve its language processing capabilities.


### With Output Processor

Different tasks require LLM to output differently. We want to provide developers with powerful choices out of box yet still allow you to customize.  

There are three types of pasers:
(1) basic string parsers, it only convert a string to a certain data type. There are float, int, and json/yaml parsers that can convert yaml or json string to yaml/json object. [String parser API reference](https://adalflow.sylph.ai/apis/core/core.string_parser.html#module-core.string_parser)

(2) Advanced output parser that will work with ``DataClass``, offering both output_format_str that instructs the LLM to output json/yaml and the ability to parse it back to structured data, either dict or ``DataClass``. [Output parser API reference](https://adalflow.sylph.ai/apis/components/components.output_parsers.outputs.html)

(3) The most hands-off ``DataClass`` parser, which can understand the ``input_fields`` and ``output_fields`` of a data class. [DataClass Parser API reference](https://adalflow.sylph.ai/apis/components/components.output_parsers.dataclass_parser.html)


Also, output parsers are just a component. You can customize it easily, and you can even chain together multiple steps of processing using ``adal.Sequential``.

#### Parser

In [ ]:
# Let LLM output json string, and convert it back to dict

output_format_str = r"""Your output should be formatted as a standard JSON object with two keys:
{
    "explanation": "A brief explanation of the concept in one sentence.",
    "example": "An example of the concept in a sentence."
}
"""

generator = adal.Generator(
    model_client=GroqAPIClient(),
    model_kwargs={"model": "llama3-8b-8192"},
    prompt_kwargs={"output_format_str": output_format_str},
    output_processors=adal.JsonParser(),
)

generator(prompt_kwargs=prompt_kwargs)


cache_path: /root/.adalflow/cache_GroqAPIClient_llama3-8b-8192.db


GeneratorOutput(id=None, data={'explanation': 'LLM stands for Large Language Model, a type of AI model that is trained on vast amounts of text data to generate human-like responses and understand natural language.', 'example': 'The new large language model was able to answer complex questions and create coherent paragraphs with ease.'}, error=None, usage=CompletionUsage(completion_tokens=62, prompt_tokens=108, total_tokens=170), raw_response='{\n"explanation": "LLM stands for Large Language Model, a type of AI model that is trained on vast amounts of text data to generate human-like responses and understand natural language.",\n"example": "The new large language model was able to answer complex questions and create coherent paragraphs with ease."\n}', metadata=None)

You can check out the `raw response`(in json string) and the parsed output at `data` which is a dict. Let's try yaml.

In [ ]:
output_format_str = r"""Your output should be formatted as a standard JSON object with two keys:
{
    "explanation": "A brief explanation of the concept in one sentence.",
    "example": "An example of the concept in a sentence."
}
"""

generator = adal.Generator(
    model_client=GroqAPIClient(),
    model_kwargs={"model": "llama3-8b-8192"},
    prompt_kwargs={"output_format_str": output_format_str},
    output_processors=adal.YamlParser(),
)

generator(prompt_kwargs=prompt_kwargs)

cache_path: /root/.adalflow/cache_GroqAPIClient_llama3-8b-8192.db


GeneratorOutput(id=None, data={'explanation': 'LLM (Large Language Model) refers to a type of artificial intelligence model that is trained on large amounts of text data to generate human-like language and understand natural language processing.', 'example': "LLMs are used in virtual assistants like Alexa and Google Assistant to understand and respond to users' voice commands."}, error=None, usage=CompletionUsage(completion_tokens=71, prompt_tokens=108, total_tokens=179), raw_response='{\n  "explanation": "LLM (Large Language Model) refers to a type of artificial intelligence model that is trained on large amounts of text data to generate human-like language and understand natural language processing.",\n  "example": "LLMs are used in virtual assistants like Alexa and Google Assistant to understand and respond to users\' voice commands."\n}', metadata=None)

The output is a yaml string, with the data in dict.

#### OutputParser

In the last example, we have to write our own output format str. We can streamline this with ``DataClass`` along with advanced output parsers: ``adal.JsonOutputParser`` and ``adal.YamlOutputParser``.

In [ ]:
from dataclasses import dataclass, field

@dataclass
class QAOutput(adal.DataClass):
    explanation: str = field(
        metadata={"desc": "A brief explanation of the concept in one sentence."}
    )
    example: str = field(metadata={"desc": "An example of the concept in a sentence."})
    question: str = field(metadata={"desc": "The question asked."}, default=None) # optional with default

    __input_fields__ = ["question"]
    __output_fields__ = ["explanation", "example"]

We need to first config the parser, this time, we want the data to be QAOutput directly.

It is not necessarily with ``__input__fields__`` and ``__output_fields__``.

Note:

*Output parser does not directly understand input or output fields, if you want to get data format str for explanation and example, you can use either `include_fields` or `exclude_fields`.*

In [ ]:
# see json format instruction asking
parser = adal.JsonOutputParser(data_class=QAOutput, return_data_class=True, include_fields=["explanation", "example"])

parser.format_instructions()

# In default, it uses signature, which is the simplified schema that simulates the json output format and field description


'Your output should be formatted as a standard JSON instance with the following schema:\n```\n{\n    "explanation": "A brief explanation of the concept in one sentence. (str) (required)",\n    "example": "An example of the concept in a sentence. (str) (required)"\n}\n```\n-Make sure to always enclose the JSON output in triple backticks (```). Please do not add anything other than valid JSON output!\n-Use double quotes for the keys and string values.\n-DO NOT mistaken the "properties" and "type" in the schema as the actual fields in the JSON output.\n-Follow the JSON formatting conventions.'

In [ ]:
from adalflow.core.base_data_class import DataClassFormatType

# the most comprehensive schema
parser.format_instructions(format_type=DataClassFormatType.SCHEMA)

'Your output should be formatted as a standard JSON instance with the following schema:\n```\n{\n    "type": "QAOutput",\n    "properties": {\n        "explanation": {\n            "type": "str",\n            "desc": "A brief explanation of the concept in one sentence."\n        },\n        "example": {\n            "type": "str",\n            "desc": "An example of the concept in a sentence."\n        }\n    },\n    "required": [\n        "explanation",\n        "example"\n    ]\n}\n```\n-Make sure to always enclose the JSON output in triple backticks (```). Please do not add anything other than valid JSON output!\n-Use double quotes for the keys and string values.\n-DO NOT mistaken the "properties" and "type" in the schema as the actual fields in the JSON output.\n-Follow the JSON formatting conventions.'

In [ ]:
# try exclude fields with question
parser = adal.JsonOutputParser(data_class=QAOutput, return_data_class=True, exclude_fields=["question"])

parser.format_instructions()

'Your output should be formatted as a standard JSON instance with the following schema:\n```\n{\n    "explanation": "A brief explanation of the concept in one sentence. (str) (required)",\n    "example": "An example of the concept in a sentence. (str) (required)"\n}\n```\n-Make sure to always enclose the JSON output in triple backticks (```). Please do not add anything other than valid JSON output!\n-Use double quotes for the keys and string values.\n-DO NOT mistaken the "properties" and "type" in the schema as the actual fields in the JSON output.\n-Follow the JSON formatting conventions.'

Note: the order of the fields in the output matters for the llm, especially when you use chain of thought, you want the rational/thought field ahead of the final prediction/answer.

If you are using OutputParser, make sure you follow exactly the order you want.

In [ ]:
def call_generator(parser):
  generator = adal.Generator(
      model_client=GroqAPIClient(),
      model_kwargs={"model": "llama3-8b-8192"},
      prompt_kwargs={"output_format_str": parser.format_instructions()},
      output_processors=parser
  )

  output = generator(prompt_kwargs=prompt_kwargs)
  print(output)

call_generator(parser)

cache_path: /root/.adalflow/cache_GroqAPIClient_llama3-8b-8192.db
GeneratorOutput(id=None, data=QAOutput(explanation='LLM stands for Large Language Model, a type of artificial intelligence algorithm that enables computers to process and understand natural language, learn from vast amounts of text data, and generate human-like responses.', example='LLMs are used in applications such as virtual assistants, chatbots, and language translation systems to provide accurate and context-specific responses.', question=None), error=None, usage=CompletionUsage(completion_tokens=80, prompt_tokens=195, total_tokens=275), raw_response='```\n{\n    "explanation": "LLM stands for Large Language Model, a type of artificial intelligence algorithm that enables computers to process and understand natural language, learn from vast amounts of text data, and generate human-like responses.",\n    "example": "LLMs are used in applications such as virtual assistants, chatbots, and language translation systems to

#### DataClassParser

Dataclass parser will understand ``__input__fields__`` and ``__output_fields__``. It's less customization and more hands-off. It uses either `yaml signature` or `json signature`.

In [ ]:
parser = adal.DataClassParser(data_class=QAOutput, return_data_class=True, format_type="yaml")

print(parser.get_input_format_str())

print(parser.get_output_format_str())


question: The question asked. (str) (optional)
output_format_str: Your output should be formatted as a standard YAML instance with the following schema:
```
explanation: A brief explanation of the concept in one sentence. (str) (required)
example: An example of the concept in a sentence. (str) (required)
```
-Make sure to always enclose the YAML output in triple backticks (```). Please do not add anything other than valid YAML output!
-Follow the YAML formatting conventions with an indent of 2 spaces.
-DO NOT mistaken the "properties" and "type" in the schema as the actual fields in the YAML output.
-Quote the string values properly.
Your output should be formatted as a standard YAML instance with the following schema:
```
explanation: A brief explanation of the concept in one sentence. (str) (required)
example: An example of the concept in a sentence. (str) (required)
```
-Make sure to always enclose the YAML output in triple backticks (```). Please do not add anything other than vali

In [ ]:
generator = adal.Generator(
      model_client=GroqAPIClient(),
      model_kwargs={"model": "llama3-8b-8192"},
      prompt_kwargs={"output_format_str": parser.get_output_format_str()},
      output_processors=parser
  )

output = generator(prompt_kwargs=prompt_kwargs)
print(output)

output_format_str: Your output should be formatted as a standard YAML instance with the following schema:
```
explanation: A brief explanation of the concept in one sentence. (str) (required)
example: An example of the concept in a sentence. (str) (required)
```
-Make sure to always enclose the YAML output in triple backticks (```). Please do not add anything other than valid YAML output!
-Follow the YAML formatting conventions with an indent of 2 spaces.
-DO NOT mistaken the "properties" and "type" in the schema as the actual fields in the YAML output.
-Quote the string values properly.
cache_path: /root/.adalflow/cache_GroqAPIClient_llama3-8b-8192.db
GeneratorOutput(id=None, data=QAOutput(explanation='LLM is a type of artificial intelligence model that uses a large-scale neural network to generate human-like language.', example='LLM models can be used to create chatbots that respond to user queries in a conversational manner.', question=None), error=None, usage=CompletionUsage(comple

#### Customize parser




In [ ]:
template = r"""<SYS>
{{system_prompt}}
</SYS>
<USER>
{{input_str}}
</USER>
"""

import re
@adal.fun_to_component
def extract_answer_string(text: str):
    """Find Answer : $value from text"""
    pattern = re.compile(r".*?Answer\s*:\s*\$?(\d+)")
    match = pattern.search(text)

    if match:
        print("match")
        return match.group(0)
    else:  # process the failure
        print("no match")
        return text

@adal.fun_to_component
def extract_answer(text: str):
    pattern = re.compile(r"Answer\s*:\s*(.*)")
    match = pattern.search(text)

    if match:
        print("match")
        return match.group(1)
    else:  # process the failure
        print("no match")
        return text

In [ ]:
output1 = extract_answer_string("Answer: $1")
print(output1)
output2 = extract_answer(output1)
print(output2)

match
Answer: $1
match
$1


In [ ]:
system_prompt = """Respond to a question in two lines (Do not use any start/end Tags):
Rationale: Your step by step reasoning.
Answer: ${answer}"""

generator = adal.Generator(
    model_client=GroqAPIClient(),
    model_kwargs={"model": "llama3-8b-8192"},
    template=template,
    prompt_kwargs={"system_prompt": system_prompt},
    output_processors=adal.Sequential(extract_answer_string, extract_answer)
)

prompt_kwargs = {"input_str": "What is LLM? Explain in one sentence."}

generator(prompt_kwargs=prompt_kwargs)

# you can also just pass extract_answer_string, its really just taking a component.

cache_path: /root/.adalflow/cache_GroqAPIClient_llama3-8b-8192.db
no match
match


GeneratorOutput(id=None, data='LLM is a type of AI model that generates human-like language responses by learning patterns and structures from a large corpus of text data.', error=None, usage=CompletionUsage(completion_tokens=65, prompt_tokens=62, total_tokens=127), raw_response='Rationale: LLM stands for Large Language Model, which is a type of AI model that is trained on a large corpus of text data to generate human-like language responses.\nAnswer: LLM is a type of AI model that generates human-like language responses by learning patterns and structures from a large corpus of text data.', metadata=None)

### Model Client Selection

AdalFlow provides an easy way to switch [`model_client`](https://adalflow.sylph.ai/tutorials/model_client.html)  in the Generator. We can even use [`ModelClientType`](https://adalflow.sylph.ai/apis/core/core.types.html#core.types.ModelClientType) to switch the model client without handling multiple imports. Let's see how both methods work:

In [ ]:
# using openai model_client

# install openai package
!pip install -U openai

clear_output()

openai_api_key = getpass("Please enter your OpenAI API key: ")


os.environ['OPENAI_API_KEY'] = openai_api_key



# import the client
from adalflow.components.model_client import OpenAIClient

generator = adal.Generator(
    model_client=OpenAIClient(),
    model_kwargs={"model": "gpt-3.5-turbo"},
)

Please enter your OpenAI API key: ··········
cache_path: /root/.adalflow/cache_OpenAIClient_gpt-3.5-turbo.db


In [ ]:
generator(prompt_kwargs=prompt_kwargs)

GeneratorOutput(id=None, data='LLM stands for Large Language Model, which is a type of artificial intelligence model that is trained on large amounts of text data to generate human-like text.', error=None, usage=CompletionUsage(completion_tokens=31, prompt_tokens=51, total_tokens=82), raw_response='LLM stands for Large Language Model, which is a type of artificial intelligence model that is trained on large amounts of text data to generate human-like text.', metadata=None)

In [ ]:
# using ModelClientType

# import the ModelClientType
from adalflow.core.types import ModelClientType

generator = adal.Generator(
    model_client=ModelClientType.OPENAI(),  # or ModelClientType.GROQ()
    model_kwargs={"model": "gpt-3.5-turbo"},
)

generator(prompt_kwargs=prompt_kwargs)

cache_path: /root/.adalflow/cache_OpenAIClient_gpt-3.5-turbo.db


GeneratorOutput(id=None, data='LLM stands for Large Language Model, a type of artificial intelligence model that is trained on vast amounts of text data to understand and generate human language.', error=None, usage=CompletionUsage(completion_tokens=30, prompt_tokens=51, total_tokens=81), raw_response='LLM stands for Large Language Model, a type of artificial intelligence model that is trained on vast amounts of text data to understand and generate human language.', metadata=None)

### Easy Creation from Configs

We can also create the generator purely from configs. There are 2 ways:
- use `from_config` method from the `Generator` class

In [ ]:
from adalflow.core import Generator

config = {
    "model_client": {
        "component_name": "GroqAPIClient",
        "component_config": {},
    },
    "model_kwargs": {
        "model": "llama3-8b-8192",
    },
}

generator: Generator = Generator.from_config(config)
# print(generator)

prompt_kwargs = {"input_str": "What is LLM? Explain in one sentence."}
# generator.print_prompt(**prompt_kwargs) # show the prompt
output = generator(
    prompt_kwargs=prompt_kwargs,
)
print(output)

cache_path: /root/.adalflow/cache_GroqAPIClient_llama3-8b-8192.db
GeneratorOutput(id=None, data='Large Language Model (LLM) is a type of artificial intelligence designed to process and generate human-like language, trained on vast amounts of text data to improve its understanding and production of language.', error=None, usage=CompletionUsage(completion_tokens=39, prompt_tokens=54, total_tokens=93), raw_response='Large Language Model (LLM) is a type of artificial intelligence designed to process and generate human-like language, trained on vast amounts of text data to improve its understanding and production of language.', metadata=None)


- create any component from configs. This is even more general. This method can be used to create any component from configs. We just need to follow the config structure: `component_name` and `component_config` for all arguments.

In [ ]:
from adalflow.utils.config import new_component
from adalflow.core import Generator

config = {
    "generator": {
        "component_name": "Generator",
        "component_config": {
            "model_client": {
                "component_name": "GroqAPIClient",
                "component_config": {},
            },
            "model_kwargs": {
                "model": "llama3-8b-8192",
            },
        },
    }
}

generator: Generator = new_component(config["generator"])
# print(generator)

prompt_kwargs = {"input_str": "What is LLM? Explain in one sentence."}
# generator.print_prompt(**prompt_kwargs)
output = generator(
    prompt_kwargs=prompt_kwargs,
)
print(output)

cache_path: /root/.adalflow/cache_GroqAPIClient_llama3-8b-8192.db
GeneratorOutput(id=None, data='LLM stands for Large Language Model, which refers to a type of artificial intelligence that is trained on vast amounts of text data to generate human-like language understanding and generation capabilities.', error=None, usage=CompletionUsage(completion_tokens=36, prompt_tokens=54, total_tokens=90), raw_response='LLM stands for Large Language Model, which refers to a type of artificial intelligence that is trained on vast amounts of text data to generate human-like language understanding and generation capabilities.', metadata=None)


It works exactly the same as the previous example. We imported Generator in this case to only show the type hinting.

## QA Chatbot

Now let's put it together and build our own QA chatbot application.

In [ ]:
class QA(adal.Component):
    def __init__(self):
        super().__init__()

        parser = adal.DataClassParser(data_class=QAOutput, return_data_class=True)
        self.generator = adal.Generator(
            model_client=GroqAPIClient(),
            model_kwargs={"model": "llama3-8b-8192"},
            template=template,
            prompt_kwargs={"output_format_str": parser.get_output_format_str()},
            output_processors=parser,
        )

    def call(self, query: str):
        return self.generator.call({"input_str": query})

    async def acall(self, query: str):
        return await self.generator.acall({"input_str": query})


qa_bot = QA()
answer = qa_bot("What is LLM?")
print(qa_bot)

output_format_str: Your output should be formatted as a standard JSON instance with the following schema:
```
{
    "explanation": "A brief explanation of the concept in one sentence. (str) (required)",
    "example": "An example of the concept in a sentence. (str) (required)"
}
```
-Make sure to always enclose the JSON output in triple backticks (```). Please do not add anything other than valid JSON output!
-Use double quotes for the keys and string values.
-DO NOT mistaken the "properties" and "type" in the schema as the actual fields in the JSON output.
-Follow the JSON formatting conventions.
cache_path: /root/.adalflow/cache_GroqAPIClient_llama3-8b-8192.db


ERROR:adalflow.components.output_parsers.dataclass_parser:Error at parsing output: Error: No JSON object or array found in the text: You're asking about LLM!

LLM stands for Large Language Model. It's a type of artificial intelligence (AI) designed to process and understand human language. These models are trained on vast amounts of text data and can generate human-like language responses, complete sentences, or even entire paragraphs.

LLMs are typically built using specialized algorithms and deep learning techniques, which allow them to learn and improve their language understanding and generation capabilities over time. They can be used in a wide range of applications, such as:

1. Language translation: LLMs can translate text from one language to another with remarkable accuracy.
2. Text summarization: They can condense long pieces of text into shorter, more digestible summaries.
3. Chatbots and customer service: LLMs can be used to power conversational interfaces, helping customer

call back on failure: GeneratorOutput(id=None, data=None, error="Error: Error: No JSON object or array found in the text: You're asking about LLM!\n\nLLM stands for Large Language Model. It's a type of artificial intelligence (AI) designed to process and understand human language. These models are trained on vast amounts of text data and can generate human-like language responses, complete sentences, or even entire paragraphs.\n\nLLMs are typically built using specialized algorithms and deep learning techniques, which allow them to learn and improve their language understanding and generation capabilities over time. They can be used in a wide range of applications, such as:\n\n1. Language translation: LLMs can translate text from one language to another with remarkable accuracy.\n2. Text summarization: They can condense long pieces of text into shorter, more digestible summaries.\n3. Chatbots and customer service: LLMs can be used to power conversational interfaces, helping customers w

In [ ]:
print(answer)

GeneratorOutput(data=QAOutput(explanation='LLM stands for Large Language Model, which refers to a type of artificial intelligence that is trained on a massive scale to generate and understand human-like language.', example='For instance, an LLM can be used to generate automated customer support responses or even help write news articles.'), error=None, usage=None, raw_response='```\n{\n    "explanation": "LLM stands for Large Language Model, which refers to a type of artificial intelligence that is trained on a massive scale to generate and understand human-like language.",\n    "example": "For instance, an LLM can be used to generate automated customer support responses or even help write news articles."\n}\n```', metadata=None)


## Examples Across the Library

Besides these examples, LLM is like water, even in our library, we have components that have adpated Generator to various other functionalities.

Checklist:
- [LLMRetriever](https://lightrag.sylph.ai/apis/components/components.retriever.llm_retriever.html#components.retriever.llm_retriever.LLMRetriever)
- [DefaultLLMJudge](https://lightrag.sylph.ai/apis/eval/eval.llm_as_judge.html#eval.llm_as_judge.DefaultLLMJudge)

## Tracing

We provide two tracing methods to help you develop and improve the Generator:
- Trace the history change (states) on prompt during your development process.
- Trace all failed LLM predictions for further improvement.

Please refer to the [tracing](https://lightrag.sylph.ai/tutorials/logging_tracing.html) to learn about these two tracing methods.

# Issues and feedback

If you encounter any issues, please report them here: [GitHub Issues](https://github.com/SylphAI-Inc/LightRAG/issues).

For feedback, you can use either the [GitHub discussions](https://github.com/SylphAI-Inc/LightRAG/discussions) or [Discord](https://discord.gg/ezzszrRZvT).